In [16]:
import gspread
import pandas as pd
import pprint

In [17]:
# give service account details to gspread
sa = gspread.service_account(filename="theta-ocean-377718-c42526ba7637.json")

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data.
copyattributes_sheet = sheet.worksheet('copyAttributes')

In [20]:
for sheets in cho_recon:
    print(sheets)

<Worksheet 'Info' id:0>
<Worksheet 'Rxns' id:1966089892>
<Worksheet 'Attributes' id:745769606>
<Worksheet 'copyAttributes' id:368082576>
<Worksheet 'Added Rxns' id:1377582373>


In [22]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(copyattributes_sheet.get_all_records())
df

,,Curated,Reaction,Reaction Formula,Subsystem,EC Number,TCDB Number,Mol wt,kcat_forward,kcat_backward,Reversible,Lower bound,Upper bound,Objective
0,0,PD,10FTHF5GLUtl,10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,0,0,1000,0
1,1,PD,10FTHF5GLUtm,10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,0,0,1000,0
2,2,PD,10FTHF6GLUtl,10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,0,0,1000,0
3,3,PD,10FTHF6GLUtm,10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,0,0,1000,0
4,4,PD,10FTHF7GLUtl,10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,0,0,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195,8195,,r2534,thr_L_e <=> thr_L_c,"TRANSPORT, EXTRACELLULAR",,,67409.13,,,1,-1000,1000,0
8196,8196,,r2535,hom_L_e <=> hom_L_c,Transport,,,67409.13,,,1,-1000,1000,0
8197,8197,,r2537,lnlncgcoa_c <=> lnlncgcoa_r,Transport,,,,,,1,-1000,1000,0
8198,8198,,r2538,dlnlcgcoa_c <=> dlnlcgcoa_r,Transport,,,,,,1,-1000,1000,0


In [ ]:
for index, row in df.iterrows():
    
    if row['Mol wt'] != '':
        row_number = index + 2
        print(row_number)
        column_index = df.columns.get_loc('Mol wt') + 1
        print(column_index)
        cell = work_sheet.cell(row_number,column_index)
        print('Cell Before Update: ',cell.value)

In [ ]:
#Update Cell
cell = work_sheet.cell(15,3)
print('Cell Before Update: ',cell.value)
work_sheet.update_cell(15,3,'N')
cell = work_sheet.cell(15,3)
print('Cell After Update: ',cell.value)

In [24]:
# add a sheet with 8200 rows and 2 columns
cho_recon.add_worksheet(rows=8200,cols=14,title='Genes')

# get the instance of the second sheet
genes_sheet = sheet.worksheet('Genes')

In [26]:
# Fetch information from the NIH database

In [370]:
from Bio import Entrez

def get_gene_info(gene_id):
    Entrez.email = 'pablodigiusto91@gmail.com'
    handle = Entrez.efetch(db='gene', id=gene_id, retmode='xml')
    record = Entrez.read(handle)[0]

    gene_name = record['Entrezgene_gene']['Gene-ref']['Gene-ref_desc']
    gene_symbol = record['Entrezgene_gene']['Gene-ref']['Gene-ref_locus']

    # check for different possible formats of the data
    if 'Entrezgene_comments' in record and 'Gene-commentary_comment' in record['Entrezgene_comments'][0]:
        gene_description = record['Entrezgene_comments'][0]['Gene-commentary_comment'][0]['String']
    elif 'Entrezgene_summary' in record:
        gene_description = record['Entrezgene_summary']
    else:
        gene_description = None

    if 'Entrezgene_track-info' in record:
        gene_ensembl_id = next((xref['Dbtag_tag']['Object-id']['Object-id_str'] for xref in record['Entrezgene_gene']['Gene-ref']['Gene-ref_db'] if xref['Dbtag_db'] == 'Ensembl'), None)
    else:
        gene_ensembl_id = None

    xrefs = record.get('Entrezgene_xtra-indexed', {}).get('Gene-commentary', [])
    print(xrefs)

    mRNA_ncbi_id = next((xref['Gene-commentary_accession'] for xref in xrefs if xref['Gene-commentary_type'].attributes['value'] == 'mRNA'), None)
    protein_ncbi_id = next((xref['Gene-commentary_accession'] for xref in xrefs if xref['Gene-commentary_type'].attributes['value'] == 'protein'), None)


    handle.close()

    return gene_symbol, gene_name, gene_description, gene_ensembl_id, mRNA_ncbi_id, protein_ncbi_id


In [373]:
gene_id = '100753375'
gene_symbol, gene_name, gene_description, gene_ensembl_id, mRNA_ncbi_id, protein_ncbi_id = get_gene_info(gene_id)
print(gene_symbol)
print(gene_name)
print(gene_description)
print(gene_ensembl_id)
print(mRNA_ncbi_id)
print(protein_ncbi_id)

KeyError: 'Entrezgene_mRNA'